In [1]:
import ROOT as R


def add_sectors(run, hn_tmpl='eid/heidindS_sample_2_2_s%d', hn_tmpl_out='h%s'):
    dir_data = '/home/ephelps/projects/phys-ana-omega/e1f6/test'
    fin = R.TFile('%s/%s_eskim.root'%(dir_data, run))
    h = fin.Get(hn_tmpl%1).Clone(hn_tmpl_out%run)
    for sect in range (2,7):
        h.Add(fin.Get(hn_tmpl%sect))
    h.SetDirectory(0)
    fin.Close()
    return h


def get_fcq(run):
    dir_data = '/home/ephelps/projects/phys-ana-omega/e1f6/test'
    fin = R.TFile('%s/%s_eskim.root'%(dir_data, run))
    lum = fin.Get('lumblocks/lumblocks')
    fcq, fcq_trip = 0, 0
    for lb in lum:
        fcq_trip += lb.fc_charge_ltcorr if lb.fc_charge_ltcorr < 0.5 else 0
        fcq += lb.fc_charge_ltcorr
    return (fcq, fcq_trip)

In [2]:
runs = ['31365_p1v1nt', '31365', '38121']
hs_e = [add_sectors(run) for run in runs]
hs_e_per_q = [h.Clone('%s_per_q'%h.GetName()) for (i,h) in enumerate(hs_e)]
for (run, h, hn) in zip(runs, hs_e, hs_e_per_q):
    h.SetTitle('eid summary, %s'%run)
    hn.Scale(1/(get_fcq(run)[1]))
    hn.SetTitle('fcq-normalized eid summary, %s'%run)

In [3]:
colors = [R.kBlue+2, R.kGreen+2, R.kRed+2]
c = R.TCanvas("ceid","EID cuts",800,600)
hs_e[0].Draw()
hs_e[0].SetLineColor(colors[0])
for i, h in enumerate(hs_e[1:], 1):
    h.Draw('same')
    h.SetLineColor(colors[i])
c.BuildLegend()
c.Draw()

In [4]:
colors = [R.kBlue+2, R.kGreen+2, R.kRed+2]
c2 = R.TCanvas("ceid2","EID cuts 2",800,600)
hs_e_per_q[0].Draw()
hs_e_per_q[0].SetLineColor(colors[0])
for i, h in enumerate(hs_e_per_q[1:], 1):
    h.Draw('same')
    h.SetLineColor(colors[i])
c2.BuildLegend()
c2.Draw()

In [3]:
runs = ['31365_p1v1', '31365', '38121']
hs_e2 = [add_sectors(run, 'eid/heid_sample_2_2_s%d', 'h%s_accum') for run in runs]
hs_e_per_q2 = [h.Clone('%s_per_q'%h.GetName()) for (i,h) in enumerate(hs_e2)]
for (run, h, hn) in zip(runs, hs_e2, hs_e_per_q2):
    h.SetTitle('cumulative. eid summary, %s'%run)
    hn.Scale(1/(get_fcq(run)[1]))
    hn.SetTitle('fcq-normalized cumulative eid summary, %s'%run)

In [4]:
colors = [R.kBlue+2, R.kGreen+2, R.kRed+2]
cc = R.TCanvas("cceid","EID c cuts",800,600)
hs_e2[0].Draw()
hs_e2[0].SetLineColor(colors[0])
for i, h in enumerate(hs_e2[1:], 1):
    h.Draw('same')
    h.SetLineColor(colors[i])
cc.BuildLegend()
cc.Draw()

In [8]:
colors = [R.kBlue+2, R.kGreen+2, R.kRed+2]
cc2 = R.TCanvas("cceid2","EID c cuts 2",800,600)
hs_e_per_q2[0].Draw()
hs_e_per_q2[0].SetLineColor(colors[0])
for i, h in enumerate(hs_e_per_q2[1:], 1):
    h.Draw('same')
    h.SetLineColor(colors[i])
cc2.BuildLegend()
cc2.Draw()

TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: cceid2


In [9]:
print(fcq38121)
print(fcq31365)

(29.92619514465332, 26.974619388580322)
(47.3635139465332, 44.75334930419922)


In [10]:
h38121n = h38121.Clone('%s_n'%(h38121.GetName()))
h31365n = h31365.Clone('%s_n'%(h31365.GetName()))
h38121n.Scale(1/fcq38121[1])
h31365n.Scale(1/fcq31365[1])

In [11]:
c2 = R.TCanvas("ceid_%d_n"%run,"EID cuts, %d, qnorm"%run,800,600)
h38121n.Draw()
h31365n.Draw('same')
h38121n.SetLineColor(R.kRed+2)
c2.Draw()